**USING LANGGRAPH**





In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

import glob
import os

embedding_function = OpenAIEmbeddings()
db_name = "vector_db"


folders = glob.glob("knowledge-base/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}


documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['doc_type'] for doc in documents)}")



# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embedding_function).delete_collection()

# Creating vectorstore

db  = Chroma.from_documents(documents=chunks, embedding=embedding_function, persist_directory=db_name)
print(f"Vectorstore created with {db._collection.count()} documents")



Created a chunk of size 557, which is longer than the specified 500
Created a chunk of size 1493, which is longer than the specified 500
Created a chunk of size 685, which is longer than the specified 500
Created a chunk of size 665, which is longer than the specified 500
Created a chunk of size 767, which is longer than the specified 500
Created a chunk of size 653, which is longer than the specified 500
Created a chunk of size 713, which is longer than the specified 500
Created a chunk of size 605, which is longer than the specified 500
Created a chunk of size 890, which is longer than the specified 500
Created a chunk of size 1031, which is longer than the specified 500
Created a chunk of size 663, which is longer than the specified 500
Created a chunk of size 706, which is longer than the specified 500
Created a chunk of size 881, which is longer than the specified 500
Created a chunk of size 1255, which is longer than the specified 500
Created a chunk of size 895, which is longer 

Total number of chunks: 136
Document types found: {'flight-information', 'safety-and-emergency-information', 'travel-insurance', 'travel-packages-and-itineraries', 'visa-and-travel-documentation', 'currency-exchange-and-budgeting', 'hotel-information', 'local-transport-and-rentals'}
Vectorstore created with 136 documents


In [3]:
collection = db._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 136 vectors with 1,536 dimensions in the vector store


In [4]:
retriever = db.as_retriever(search_kwargs={"k": 6})

In [5]:
from langchain_core.prompts import ChatPromptTemplate


template = """You're a lively and witty travel assistant, always ready for a friendly chat! Your goal is to provide concise (1-2 sentence) answers that are informative, humorous, and engaging. Feel free to sprinkle in light jokes or travel tips where appropriate.

When responding, follow these guidelines:
- Prioritize the latest question while considering relevant context and chat history.
- Be polite, positive, and enthusiastic.
- Offer practical suggestions with a cheerful tone.
- Encourage further conversation with follow-up questions.
- If the response involves multiple recommendations, options, or explanations, present them using fun emojis or creative separators like 🚀, 🌟, or ✅ for clarity and engagement.

Chathistory: {history}
Context: {context}
Question: {question}

Respond cheerfully and engagingly:
"""


prompt = ChatPromptTemplate.from_template(template)

In [6]:
llm = ChatOpenAI(model="gpt-4o-mini")

rag_chain = prompt | llm

In [7]:
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain.schema import Document
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END


class AgentState(TypedDict):
    messages: List[BaseMessage]
    documents: List[Document]
    on_topic: str
    rephrased_question: str
    proceed_to_generate: bool
    rephrase_count: int
    question: HumanMessage


class GradeQuestion(BaseModel):
    score: float = Field(description="Confidence score that the question is about the specified topics. Ranges from 0.0 to 1.0.")


def question_rewriter(state: AgentState):
    print(f"Entering question_rewriter with following state: {state}")

    state["documents"] = []
    state["on_topic"] = ""
    state["rephrased_question"] = ""
    state["proceed_to_generate"] = False
    state["rephrase_count"] = 0

    if "messages" not in state or state["messages"] is None:
        state["messages"] = []

    if state["question"] not in state["messages"]:
        state["messages"].append(state["question"])

    if len(state["messages"]) > 1:
        conversation = state["messages"][:-1]
        current_question = state["question"].content
        messages = [
            SystemMessage(
                # content="You are a helpful assistant that rephrases the user's question to be a standalone question optimized for retrieval."
                content="You are a friendly, witty, and engaging travel assistant who loves chatting with users! You rephrase questions in a natural, conversational way while keeping them informative and useful."

            )
        ]
        messages.extend(conversation)
        messages.append(HumanMessage(content=current_question))
        rephrase_prompt = ChatPromptTemplate.from_messages(messages)
        llm = ChatOpenAI(model="gpt-4o-mini")
        prompt = rephrase_prompt.format()
        response = llm.invoke(prompt)
        better_question = response.content.strip()
        print(f"question_rewriter: Rephrased question: {better_question}")
        state["rephrased_question"] = better_question
    else:
        state["rephrased_question"] = state["question"].content
    return state


def question_classifier(state: AgentState):
    print("Entering question_classifier")

    system_message = SystemMessage(
        content="""You are a classifier that determines whether a user's question is about one of the following topics:

        1. Flight Information, including booking policies, baggage rules, cancellations, refunds, rescheduling, no-show & missed flight policies, and travel class upgrades.
        2. Hotel Information, including booking guidelines, cancellations, deposits, extra charges, peak season pricing, and refund policies.
        3. Visa and Travel Documentation, including visa requirements, application processes, rejection reasons, and necessary travel documents.
        4. Travel Packages and Itineraries, including package details, popular destinations, sample itineraries, pricing, inclusions, and insurance recommendations.
        5. Local Transport and Rentals, including airport transfers, public transport, car rentals, and ride-sharing options.
        6. Travel Insurance, including types of coverage, exclusions, and claims processes.
        7. Safety and Emergency Information, including emergency contacts, common travel scams, health advisories, and safety tips.
        8. Currency Exchange and Budgeting, including exchange options, rate tracking, ATM withdrawals, budgeting, and money safety tips.
        
        Provide a confidence score from 0.0 to 1.0. A higher score means higher confidence that the question is on one of these topics."""
    )
    

    human_message = HumanMessage(
        content=f"User question: {state['rephrased_question']}"
    )
    grade_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    llm = ChatOpenAI(model="gpt-4o-mini")
    structured_llm = llm.with_structured_output(GradeQuestion)
    grader_llm = grade_prompt | structured_llm
    result = grader_llm.invoke({})
    state["on_topic"] = result.score
    print(f"question_classifier: on_topic = {state['on_topic']}")
    return state

def on_topic_router(state: AgentState):
    print("Entering on_topic_router")
    confidence_score = state.get("on_topic", 0.0)

    if confidence_score >= 0.5:
        print(f"Routing to retrieve with Confidence Score: {confidence_score}")
        return "retrieve"
    else:
        print(f"Routing to off_topic_response with Confidence Score: {confidence_score}")
        return "off_topic_response"


def retrieve(state: AgentState):
    print("Entering retrieve")
    documents = retriever.invoke(state["rephrased_question"])
    print(f"retrieve: Retrieved {len(documents)} documents")
    state["documents"] = documents
    return state


class GradeDocument(BaseModel):
    score: str = Field(
        description="Document is relevant to the question? If yes -> 'Yes' if not -> 'No'"
    )


def retrieval_grader(state: AgentState):
    print("Entering retrieval_grader")
    system_message = SystemMessage(
        content="""You are a grader assessing the relevance of a retrieved document to a user question.
Only answer with 'Yes' or 'No'.

If the document contains information relevant to the user's question, respond with 'Yes'.
Otherwise, respond with 'No'."""
    )

    llm = ChatOpenAI(model="gpt-4o-mini")
    structured_llm = llm.with_structured_output(GradeDocument)

    relevant_docs = []
    for doc in state["documents"]:
        human_message = HumanMessage(
            content=f"User question: {state['rephrased_question']}\n\nRetrieved document:\n{doc.page_content}"
        )
        grade_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
        grader_llm = grade_prompt | structured_llm
        result = grader_llm.invoke({})
        print(
            f"Grading document: {doc.page_content[:30]}... Result: {result.score.strip()}"
        )
        if result.score.strip().lower() == "yes":
            relevant_docs.append(doc)
    state["documents"] = relevant_docs
    state["proceed_to_generate"] = len(relevant_docs) > 0
    print(f"retrieval_grader: proceed_to_generate = {state['proceed_to_generate']}")
    return state


def proceed_router(state: AgentState):
    print("Entering proceed_router")
    rephrase_count = state.get("rephrase_count", 0)
    if state.get("proceed_to_generate", False):
        print("Routing to generate_answer")
        return "generate_answer"
    elif rephrase_count >= 2:
        print("Maximum rephrase attempts reached. Cannot find relevant documents.")
        return "cannot_answer"
    else:
        print("Routing to refine_question")
        return "refine_question"


def refine_question(state: AgentState):
    print("Entering refine_question")
    rephrase_count = state.get("rephrase_count", 0)
    if rephrase_count >= 2:
        print("Maximum rephrase attempts reached")
        return state
    question_to_refine = state["rephrased_question"]
    system_message = SystemMessage(
        content="""You are a helpful assistant that slightly refines the user's question to improve retrieval results.
Provide a slightly adjusted version of the question."""
    )
    human_message = HumanMessage(
        content=f"Original question: {question_to_refine}\n\nProvide a slightly refined question."
    )
    refine_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = refine_prompt.format()
    response = llm.invoke(prompt)
    refined_question = response.content.strip()
    print(f"refine_question: Refined question: {refined_question}")
    state["rephrased_question"] = refined_question
    state["rephrase_count"] = rephrase_count + 1
    return state


import random

def generate_answer(state: AgentState):
    print("Entering generate_answer")
    if "messages" not in state or state["messages"] is None:
        raise ValueError("State must include 'messages' before generating an answer.")

    history = state["messages"]
    documents = state["documents"]
    rephrased_question = state["rephrased_question"]

    response = rag_chain.invoke(
        {"history": history, "context": documents, "question": rephrased_question}
    )

    response_text = response.content.strip()

    # Fun, engaging follow-up lines
    fun_replies = [
        "Hope that helps! Need more travel tips? 😊",
        "That’s the scoop! Got any other questions? ✈️",
        "Wanna know more? Just ask! 🌍",
        "If you’re curious about anything else, I’m all ears! 👂",
        "Travel talk is my jam! Hit me up with another question. 🚀"
    ]

    # Add a random fun reply to the response
    generation = f"{response_text} {random.choice(fun_replies)}"

    state["messages"].append(AIMessage(content=generation))
    print(f"generate_answer: Generated response: {generation}")
    return state


def cannot_answer(state: AgentState):
    print("Entering cannot_answer")
    if "messages" not in state or state["messages"] is None:
        state["messages"] = []
    state["messages"].append(
        AIMessage(
            content="I'm sorry, but I cannot find the information you're looking for."
        )
    )
    return state


def off_topic_response(state: AgentState):
    print("Entering off_topic_response")
    if "messages" not in state or state["messages"] is None:
        state["messages"] = []
    state["messages"].append(AIMessage(content="Oof, I wish I could help with that! But I’m all about travel. Wanna ask me about flights, hotels, or dreamy destinations instead? 😎✈️"))
    return state

In [8]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()

In [9]:
# Workflow
workflow = StateGraph(AgentState)
workflow.add_node("question_rewriter", question_rewriter)
workflow.add_node("question_classifier", question_classifier)
workflow.add_node("off_topic_response", off_topic_response)
workflow.add_node("retrieve", retrieve)
workflow.add_node("retrieval_grader", retrieval_grader)
workflow.add_node("generate_answer", generate_answer)
workflow.add_node("refine_question", refine_question)
workflow.add_node("cannot_answer", cannot_answer)

workflow.add_edge("question_rewriter", "question_classifier")
workflow.add_conditional_edges(
    "question_classifier",
    on_topic_router,
    {
        "retrieve": "retrieve",
        "off_topic_response": "off_topic_response",
    },
)
workflow.add_edge("retrieve", "retrieval_grader")
workflow.add_conditional_edges(
    "retrieval_grader",
    proceed_router,
    {
        "generate_answer": "generate_answer",
        "refine_question": "refine_question",
        "cannot_answer": "cannot_answer",
    },
)
workflow.add_edge("refine_question", "retrieve")
workflow.add_edge("generate_answer", END)
workflow.add_edge("cannot_answer", END)
workflow.add_edge("off_topic_response", END)
workflow.set_entry_point("question_rewriter")
graph = workflow.compile(checkpointer=checkpointer)

In [11]:
# from IPython.display import Image, display
# from langchain_core.runnables.graph import MermaidDrawMethod

# display(
#     Image(
#         graph.get_graph().draw_mermaid_png(
#             draw_method=MermaidDrawMethod.API,
#         )
#     )
# )

In [12]:
# input_data = {
#     "question": HumanMessage(content="what are the popular destinations to visit?")
# }
# graph.invoke(input=input_data, config={"configurable": {"thread_id": 1}})

In [13]:
# input_data = {"question": HumanMessage(content="in that suggest me one?")}
# graph.invoke(input=input_data, config={"configurable": {"thread_id": 1}})

In [14]:

import re

user_name = None

def extract_name(question):

    patterns = [
        r"\bmy name is (\w+)\b",
        r"\bi am (\w+)\b",
        r"\bthis is (\w+)\b",
        r"\bcall me (\w+)\b",
        r"\bi'm (\w+)\b",
        r"\bthey call me (\w+)\b",
        r"\bthe name's (\w+)\b",
        r"\bname me (\w+)\b",
        r"\bjust call me (\w+)\b",
        r"\bpeople call me (\w+)\b",
        r"\byou can call me (\w+)\b",
        r"\bhello, (\w+)\b",
        r"\bhey, (\w+)\b",
        r"\bhi, i'm (\w+)\b",
        r"\bit's (\w+)\b",
        r"\bmy friends call me (\w+)\b",
        r"\bknown as (\w+)\b",
        r"^(\w+)$"
    ]

    for pattern in patterns:
        match = re.search(pattern, question, re.IGNORECASE)
        if match:
            return match.group(1)

    return None


def process_question(question: str, thread_id: int = 1):
    global user_name
    
    # Check for greetings
    if any(greet in question.lower() for greet in ["hi", "hello", "hey"]):
        if user_name:
            return f"Hey {user_name}! Welcome back! What’s on your travel wishlist today? 🌍✈️"
        return "Hey there! What should I call you? 😊"
    
    # Check if user introduced themselves
    name = extract_name(question)
    if name:
        user_name = name
        return f"Nice to meet you, {user_name}! Ask me anything about travel. 🌍✈️"

    input_data = {"question": HumanMessage(content=question)}
    result = graph.invoke(input=input_data, config={"configurable": {"thread_id": thread_id}})
    
    messages = result.get("messages", [])
    ai_responses = [msg.content for msg in messages if isinstance(msg, AIMessage)]
    
    return ai_responses[-1] if ai_responses else "Sorry, I couldn't find an answer."


**version 0 for gradio UI - works well**

In [15]:

# import gradio as gr

# def gradio_interface(question):
#     """
#     Gradio chat interface function to process user questions.
#     """
#     return process_question(question)

# # Create Gradio Chat UI
# with gr.Blocks(theme=gr.themes.Soft()) as ui:
#     gr.Markdown("# ✈️ Travel Assistant")
#     gr.Markdown("Ask me anything about flights, hotels, visas, travel packages, and more!")

#     chatbot = gr.Chatbot(label="Travel Assistant", type="messages")
#     msg = gr.Textbox(placeholder="Type your question here...", label="Your Question")

#     def chat_function(messages, message):
#         response = gradio_interface(message)
#         messages.append({"role": "user", "content": message})
#         messages.append({"role": "assistant", "content": response})
#         return messages, ""

#     msg.submit(chat_function, [chatbot, msg], [chatbot, msg])

# # Launch UI
# ui.launch()


**Testing Image Generation - works well**

In [16]:
# import base64
# from io import BytesIO
# from PIL import Image

In [17]:
# def artist(city):
#     image_response = openai.images.generate(
#             model="dall-e-3",
#             prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
#             size="1024x1024",
#             n=1,
#             response_format="b64_json",
#         )
#     image_base64 = image_response.data[0].b64_json
#     image_data = base64.b64decode(image_base64)
#     return Image.open(BytesIO(image_data))

In [18]:
# image = artist("india")
# display(image)

**Testing text to speech for the responses - works well**

In [19]:
# import base64
# from io import BytesIO
# from PIL import Image
# from IPython.display import Audio, display

# def talker(message):
#     response = openai.audio.speech.create(
#         model="tts-1",
#         voice="onyx",
#         input=message)

#     audio_stream = BytesIO(response.content)
#     output_filename = "output_audio.mp3"
#     with open(output_filename, "wb") as f:
#         f.write(audio_stream.read())

#     # Play the generated audio
#     display(Audio(output_filename, autoplay=True))

# talker("Well, hi madhan how are you. its been so long we met each other. nice to meet you..")

**version 1 Merging the image and audio generation with gradio ui**

In [20]:
# import gradio as gr
# import base64
# from io import BytesIO
# from PIL import Image
# from IPython.display import Audio, display
# from openai import OpenAI
# import requests

# # OpenAI API client
# client = OpenAI()

# # Track whether an image should be generated
# image_context = None
# generated_image = None


# def generate_image(prompt):
#     global image_context, generated_image
    
#     # If the context hasn't changed, return the existing image
#     if prompt == image_context and generated_image is not None:
#         return generated_image
    
#     # Otherwise, generate a new image
#     image_response = client.images.generate(
#         model="dall-e-3",
#         # prompt=f"A travel-themed image representing {prompt}, featuring landmarks, nature, and vibrant scenery.",
#         prompt=f"A modern comic book panel illustration of a travel-themed scene representing {prompt}, featuring landmarks, cityscapes, or nature with vibrant colors, bold outlines, and dynamic storytelling. Characters interacting with the environment, detailed textures, and expressive poses. Highly detailed with shading and comic-like dialogue elements.",
#         size="1024x1024",
#         n=1,
#         response_format="b64_json",
#     )
    
#     image_base64 = image_response.data[0].b64_json
#     image_data = base64.b64decode(image_base64)
    
#     # Update the context and store the image
#     image_context = prompt
#     generated_image = Image.open(BytesIO(image_data))
#     return generated_image


# def generate_audio(text):
#     response = client.audio.speech.create(
#         model="tts-1",
#         voice="onyx",
#         input=text
#     )
    
#     audio_stream = BytesIO(response.content)
#     output_filename = "output_audio.mp3"
#     with open(output_filename, "wb") as f:
#         f.write(audio_stream.read())

#     return output_filename


# def chat_with_options(messages, message, image_toggle, audio_toggle):
#     """
#     Handles chat input and generates responses.
#     Also triggers image and audio generation if enabled.
#     """
#     global generated_image
    
#     response_text = process_question(message)  # Call the main chatbot function
    
#     image_output = None
#     audio_output = None
    
#     if image_toggle:
#         image_output = generate_image(message)

#     if audio_toggle:
#         audio_output = generate_audio(response_text)
    
#     # Append chat messages
#     messages.append({"role": "user", "content": message})
#     messages.append({"role": "assistant", "content": response_text})

#     return messages, "", image_output, audio_output


# # Gradio UI with two sections
# with gr.Blocks(theme=gr.themes.Soft()) as ui:
#     gr.Markdown("# ✈️ Travel Assistant")
#     gr.Markdown("Ask me anything about flights, hotels, visas, travel packages, and more!")

#     with gr.Row():  # Split UI
#         with gr.Column(scale=2):  # Chat section
#             chatbot = gr.Chatbot(label="Travel Assistant", type="messages")
#             msg = gr.Textbox(placeholder="Type your question here...", label="Your Question")
#             with gr.Row():
#                 img_toggle = gr.Checkbox(label="Generate Image")
#                 audio_toggle = gr.Checkbox(label="Enable Audio Response")

#         with gr.Column(scale=1):  # Image display
#             image_display = gr.Image(label="Generated Image", type="pil")

#     # Audio output
#     audio_output = gr.Audio(label="Response Audio", autoplay=True)

#     # Chat function with toggles
#     msg.submit(chat_with_options, [chatbot, msg, img_toggle, audio_toggle], [chatbot, msg, image_display, audio_output])

# # Launch UI
# ui.launch()


**version 2 for merging audio and video with Gradio UI**

In [21]:
# import gradio as gr
# import base64
# import os
# import time
# from io import BytesIO
# from PIL import Image
# from openai import OpenAI
# import pygame

# # Initialize OpenAI API client
# client = OpenAI()

# # Create directories for saving images and audio files
# os.makedirs("image_files", exist_ok=True)
# os.makedirs("audio_files", exist_ok=True)

# # Track generated image to prevent unnecessary re-generation
# image_context = None
# generated_image = None

# def get_timestamp():
#     """Generate a timestamp for unique file naming."""
#     return time.strftime("%d-%m-%Y-%H-%M-%S")

# def generate_image(prompt):
#     """Generate and save an image based on the user's prompt."""
#     global image_context, generated_image

#     if prompt == image_context and generated_image is not None:
#         return generated_image

#     # Generate new image from OpenAI
#     image_response = client.images.generate(
#         model="dall-e-3",
#         prompt=f"A modern comic book panel illustration of a travel-themed scene representing {prompt}, featuring landmarks, cityscapes, or nature with vibrant colors, bold outlines, and dynamic storytelling. Characters interacting with the environment, detailed textures, and expressive poses. Highly detailed with shading and comic-like dialogue elements.",
#         size="1024x1024",
#         n=1,
#         response_format="b64_json",
#     )

#     image_base64 = image_response.data[0].b64_json
#     image_data = base64.b64decode(image_base64)
    
#     # Save image with unique filename
#     timestamp = get_timestamp()
#     image_path = f"image_files/travel_image_{timestamp}.png"
#     with open(image_path, "wb") as f:
#         f.write(image_data)
    
#     image_context = prompt
#     generated_image = Image.open(image_path)
#     return generated_image

# def generate_audio(text):
#     """Generate and save audio, then play it in the background."""
#     response = client.audio.speech.create(
#         model="tts-1",
#         voice="alloy",
#         input=text
#     )
    
#     timestamp = get_timestamp()
#     audio_path = f"audio_files/response_audio_{timestamp}.mp3"
    
#     with open(audio_path, "wb") as f:
#         f.write(response.content)
    
#     # Play audio in background
#     pygame.mixer.init()
#     pygame.mixer.music.load(audio_path)
#     pygame.mixer.music.play()
    
#     return audio_path  # Returning for reference if needed, but not displaying

# def chat_with_options(messages, message, image_toggle, audio_toggle):
#     """Handles chat and triggers image and audio generation if enabled."""
#     global generated_image

#     response_text = process_question(message)  # Replace with your chatbot logic
    
#     image_output = None

#     if image_toggle:
#         image_output = generate_image(message)

#     if audio_toggle:
#         generate_audio(response_text)  # Play audio in the background

#     messages.append({"role": "user", "content": message})
#     messages.append({"role": "assistant", "content": response_text})

#     return messages, "", image_output  # Remove audio_output as it plays in the background

# # Gradio UI
# with gr.Blocks(theme=gr.themes.Soft()) as ui:
#     gr.Markdown("# ✈️ Travel Assistant")
#     gr.Markdown("Ask me anything about flights, hotels, visas, travel packages, and more!")

#     with gr.Row():  
#         with gr.Column(scale=2):  
#             chatbot = gr.Chatbot(label="Travel Assistant", type="messages")
#             msg = gr.Textbox(placeholder="Type your question here...", label="Your Question")
#             with gr.Row():
#                 img_toggle = gr.Checkbox(label="Generate Image")
#                 audio_toggle = gr.Checkbox(label="Enable Audio Response")

#         with gr.Column(scale=1):  
#             image_display = gr.Image(label="Generated Image", type="pil")

#     msg.submit(chat_with_options, [chatbot, msg, img_toggle, audio_toggle], [chatbot, msg, image_display])

# ui.launch()


In [22]:
!pip install pygame


**version 3 for merging audio and image with gradio ui**

In [ ]:
import gradio as gr
import base64
import os
import time
from io import BytesIO
from PIL import Image
from openai import OpenAI
import pygame

# Initialize OpenAI API client
client = OpenAI()

# Create directories for saving images and audio files
os.makedirs("image_files", exist_ok=True)
os.makedirs("audio_files", exist_ok=True)

# Track generated image and context
image_context = None
generated_image = None

def get_timestamp():
    """Generate a timestamp for unique file naming."""
    return time.strftime("%d-%m-%Y-%H-%M-%S")

def is_same_context(prev_prompt, new_prompt):
    """Determine if two prompts belong to the same context using OpenAI embeddings."""
    if not prev_prompt:
        return False  # No previous context, treat as new

    embedding_response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=[prev_prompt, new_prompt]
    )
    embeddings = embedding_response.data
    similarity_score = sum(a * b for a, b in zip(embeddings[0].embedding, embeddings[1].embedding))
    
    return similarity_score > 0.9  # Adjust threshold as needed

def generate_image(prompt):
    """Generate and save an image based on the user's prompt."""
    global image_context, generated_image

    if image_context and is_same_context(image_context, prompt):
        return generated_image  # Keep the previous image if context is the same

    # Generate new image from OpenAI
    image_response = client.images.generate(
        model="dall-e-3",
        prompt=f"A modern comic book panel illustration of a travel-themed scene representing {prompt}, featuring landmarks, cityscapes, or nature with vibrant colors, bold outlines, and dynamic storytelling.",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )

    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    
    # Save image with unique filename
    timestamp = get_timestamp()
    image_path = f"image_files/travel_image_{timestamp}.png"
    with open(image_path, "wb") as f:
        f.write(image_data)
    
    image_context = prompt
    generated_image = Image.open(image_path)
    return generated_image

def generate_audio(text):
    """Generate and save audio, then play it in the background."""
    response = client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=text
    )
    
    timestamp = get_timestamp()
    audio_path = f"audio_files/response_audio_{timestamp}.mp3"
    
    with open(audio_path, "wb") as f:
        f.write(response.content)
    
    # Play audio in background
    pygame.mixer.init()
    pygame.mixer.music.load(audio_path)
    pygame.mixer.music.play()
    
    return audio_path  # Returning for reference if needed, but not displaying

def chat_with_options(messages, message, image_toggle, audio_toggle):
    """Handles chat and triggers image and audio generation if enabled."""
    global generated_image

    response_text = process_question(message)  # Replace with your chatbot logic
    
    image_output = None
    if generated_image and is_same_context(image_context, message):
        image_output = generated_image  # Keep previous image
    elif image_toggle:
        image_output = generate_image(message)  # Generate new image only if enabled

    if audio_toggle:
        generate_audio(response_text)  # Play audio in the background

    messages.append({"role": "user", "content": message})
    messages.append({"role": "assistant", "content": response_text})

    return messages, "", image_output  # Remove audio_output as it plays in the background

# Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as ui:
    gr.Markdown("# ✈️ Travel Assistant")
    gr.Markdown("Ask me anything about flights, hotels, visas, travel packages, and more!")

    with gr.Row():  
        with gr.Column(scale=2):  
            chatbot = gr.Chatbot(label="Travel Assistant", type="messages")
            msg = gr.Textbox(placeholder="Type your question here...", label="Your Question")
            with gr.Row():
                img_toggle = gr.Checkbox(label="Generate Image")
                audio_toggle = gr.Checkbox(label="Enable Audio Response")

        with gr.Column(scale=1):  
            image_display = gr.Image(label="Generated Image", type="pil")

    msg.submit(chat_with_options, [chatbot, msg, img_toggle, audio_toggle], [chatbot, msg, image_display])

ui.launch(share=True, debug = True)


pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
